In [1]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [7]:
#Load trained model,scaler pickle,ohe
model = load_model('/content/drive/MyDrive/ML DLprojects/1ANNimplementation-classification/models.h5')

#Load encoder ans scaler :
with open('/content/drive/MyDrive/ML DLprojects/1ANNimplementation-classification/label_encoder_gender.pkl','rb') as file:
  label_encoder_gender = pickle.load(file)

with open('/content/drive/MyDrive/ML DLprojects/1ANNimplementation-classification/ohe_geo.pkl','rb') as file:
  ohe_geo = pickle.load(file)

with open('/content/drive/MyDrive/ML DLprojects/1ANNimplementation-classification/scaler.pkl','rb') as file:
  scaler = pickle.load(file)

In [8]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [17]:
geo_encoded = ohe_geo.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded,columns=ohe_geo.get_feature_names_out(['Geography']))

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [18]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [19]:
input_df = pd.DataFrame([input_data])

In [20]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [21]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])

In [22]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [24]:
input_df = pd.concat([input_df.drop(columns=['Geography']),geo_encoded_df],axis=1)

In [25]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [26]:
#Scaling the input
input_scaled = scaler.transform(input_df)

In [27]:
input_scaled

array([[-0.52205552,  0.91647499,  0.09143332, -0.6884345 , -0.27743646,
         0.80485159,  0.64920267,  0.96462528, -0.81749466,  0.99104014,
        -0.55964063, -0.5881284 ]])

In [28]:
#Predict churn
prediction = model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step


In [29]:
prediction

array([[0.0740374]], dtype=float32)

In [32]:
prediction_prob = prediction[0][0]

In [33]:
prediction_prob

np.float32(0.074037395)

In [35]:
if prediction_prob > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
